In [1]:
import sys
import pandas as pd
import csv
import numpy as np
import collections
import os
import re
import jieba
import string
import operator

In [2]:
os.chdir('D:/bigdatahw/应用统计案例/case2018Lab9')    

- 导进去的文档是按照时间顺序从早到晚排列，也就是说，df中最开始的是4月17日的文档，最末尾的是5月19日的文档。下面的clean_text也是按顺序的，这一点对之后建立动态主题模型很重要。

In [3]:
df = pd.read_csv("alldata.csv",encoding = "gb18030")
df.head(3)

,title,date,time,source,abstract,detail,url,origin_url,period
0,"通源石油:中美贸易战对公司利大于弊,油价复苏带动业绩大增",2018/3/28,15:34:07,中证网,"事件:公司3月26日发布关于美贸易政策影响的说明公告,认为中美贸易战对公司的营业收入和日常经...","事件:公司3月26日发布关于美贸易政策影响的说明公告,认为中美贸易战对公司的营业收入和日常经...",http://finance.sina.com.cn/roll/2018-03-28/doc...,http://www.cs.com.cn/gppd/tzpj/201803/t2018032...,1
1,人民日报海外版:美国一意孤行 世界一声叹息,2018/3/29,5:22:07,人民日报海外版-海外网,欧洲股市、亚太股市同样无法幸免。“中美贸易战还未真正开打，全球股市已经出现大幅下挫，这就足以...,原标题：美国一意孤行 世界一声叹息（环球热点）美国特朗普政府掀起的贸易保护阴云弥漫不散。3...,http://news.sina.com.cn/c/nd/2018-03-29/doc-if...,http://news.haiwainet.cn/n/2018/0329/c3541083-...,1
2,分析：中美贸易关系将如何影响国内外豆粕行情？,2018/3/29,8:13:42,农产品期货网,00:40分左右，美国总统特朗普正式签署了制裁中国的总统备忘录，美国政府计划对中国的至少60...,?? 3月初以来芝商所旗下CBOT期价（代码：ZMK8）一度陷入震荡回落囧境，最低曾下探至...,http://finance.sina.com.cn/money/future/agri/2...,http://www.ncpqh.com/news/getDetail?newsclass=...,1


- 不同日期的新闻数

In [4]:
df["date"]= pd.to_datetime(df["date"],format="%Y-%m-%d")
df["date"].value_counts(sort = True)

2018-04-09    90
2018-04-10    60
2018-04-06    44
2018-04-11    39
2018-04-05    37
2018-04-08    35
2018-04-18    29
2018-04-07    28
2018-04-17    27
2018-04-13    22
2018-05-08    14
2018-05-15    13
2018-04-02    13
2018-04-27    12
2018-04-19    12
2018-04-04    11
2018-04-24    11
2018-04-23    10
2018-04-12     9
2018-05-02     9
2018-04-03     7
2018-05-07     6
2018-05-14     5
2018-05-03     5
2018-04-22     5
2018-05-20     5
2018-05-04     5
2018-04-25     4
2018-03-31     4
2018-03-30     4
2018-05-10     4
2018-04-21     3
2018-05-16     3
2018-05-18     3
2018-05-01     3
2018-04-15     2
2018-05-11     2
2018-04-28     2
2018-04-16     2
2018-03-29     2
2018-05-05     2
2018-03-28     1
2018-04-01     1
2018-05-09     1
2018-05-17     1
2018-04-14     1
2018-04-26     1
2018-05-19     1
Name: date, dtype: int64

In [5]:
from collections import Counter,defaultdict
import os
import pandas as pd
import re
import jieba
import numpy as np
import operator

def get_text(data):
    text=data["detail"]
    text = text.dropna() 
    len(text)
    text=[t.encode('utf-8').decode("utf-8") for t in text] 
    return text
def get_stop_words(file='stopWord.txt'):
    file = open(file, 'rb').read().decode('utf8').split(',')
    file = [line.strip() for line in file]
    return set(file)                                         #查分停用词函数


def rm_tokens(words):                                        # 去掉一些停用词和完全包含数字的字符串
    words_list = list(words)
    stop_words = get_stop_words()
    for i in range(words_list.__len__())[::-1]:
        if words_list[i] in stop_words:                      # 去除停用词
            words_list.pop(i)
        elif words_list[i].isdigit():
            words_list.pop(i)
    return words_list



def rm_char(text):

    text = re.sub('\x01', '', text)                        #全角的空白符
    text = re.sub('\u3000', '', text) 
    text = re.sub(r"[\)(↓%·▲ \s+】【]","", text.encode("utf-8").decode("utf-8")) 
    text = re.sub(r"[\n\da-z+（）《》↗><‘’“”.-]","",text,flags=re.I)
    text = re.sub('\n+', " ", text)
    text = re.sub('[，、：。！？?；——]', " ", text)
#    text = re.sub(' +', "E S", text)
    return text

def convert_doc_to_wordlist(str_doc, cut_all=False):
    sent_list = str_doc.split('\n')
    sent_list = map(rm_char, sent_list)                       # 去掉一些字符，例如\u3000
    word_2dlist = [rm_tokens(jieba.cut(part, cut_all=cut_all))
                   for part in sent_list]                     # 分词
    word_list = sum(word_2dlist, [])
    rm_space = [w for w in word_list if w not in [""," "]]
    return rm_space

def rm_1ow_freq_word(texts,low_freq=1):
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    
    texts = [[token for token in text if frequency[token] > low_freq] for text in texts]
    return texts

jieba.add_word("中美")
jieba.add_word("中美贸易战")
jieba.add_word("习主席")
jieba.add_word("中国商务部")
jieba.add_word("拒绝令")
jieba.add_word("不公平")
jieba.add_word("推文")
jieba.add_word("推特")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BOKKIN~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built succesfully.


In [ ]:
clean_text=[convert_doc_to_wordlist(line) for line in get_text(df)] #去除停用词、标点、数字、特殊字符、分词
clean_text2 = rm_1ow_freq_word(clean_text)#去除只出现过一次的词

In [8]:
"/".join(clean_text2[302])

'上/交易日/市况/回顾/周一/美元/竞对/货币/小幅/走低/此前/美元/连续/两周/上涨/投资者/解除/一些/美元/空头/押注/贸易战/忧虑/依然/存在/这令/美元/整体/承压/欧元/欧洲央行/行长/德拉/基/讲话/后/上涨/市场/认为/德拉/基的/讲话/欧元/起到/支撑/作用/大宗/商品/股市/方面/贵金属/小幅/收涨/国际/油价/收涨/逾/欧美/股市/涨跌/不/交易日/市场/重点/关注/中国/国家/主席/习近平/博鳌/亚洲/论坛/主旨/演讲/美国/数据/外汇/欧元/美元/收报/英镑/美元/收报/澳元/美元/收报/美元/日元/收报/美元/加元/收报/美元/瑞郎/收报/大宗/商品/现货/金/收报/美元/盎司/期金/收报/美元/盎司/现货/银/收报/美元/盎司/期银/收报/美元/盎司/布伦特/收报/美元/桶/原油/收报/美元/桶/欧美/股市/周一/美市/盘中/最后/一个/小时/亚马逊/波音/翻盘/美股/急转直下/道/指点/涨势/几乎/回吐/殆尽/股市/收盘/远/低于/盘中/高点/交易员/表示/一份/官员/突击/搜查/美国/总统/特朗普/私人/律师/办公室/报告/加剧/尾盘/跌势/道琼斯/工业/平均/指数/上涨/点/,/点/默克/英特尔/指数/中/表现/最好/股票/标准/普尔/指数/上涨/收于/,/点/科技/板块/大涨/纳斯达克/综合/指数/上涨/至点/欧洲/股市/周一/小幅/收高/因在/周二/关键/讲话/前/市场/中美贸易战/担忧/略有/减弱/泛欧/斯托克/指数/徘徊/平盘/附近/收盘/仅/上涨/板块/不同/方向/移动/当天/早些时候/指数/一直/上涨/汇市/上/交易日/纽约/时段/欧元/美元/开于/时段/内/窄幅/震荡/尾盘/交投/附近/技术/面看/绿色/动能/柱/稳步/收缩/指标/指向/中线/上方/预示/汇价/下/行动/正在/消退/汇价/上行/初步/阻力/进一步/阻力/位于/更/关键/阻力/位于/汇价/下行/初步/支撑/位于/进一步/支撑/位于/更/关键/支撑/位于/英镑/美元/时段/开于/时段/内/小幅/下挫/尾盘/交投/附近/技术/面看/红色/动能/柱/显现/指标/低位/向上/进入/超买/区域/显示/汇价/上行/势头/依然/汇价/上行/初步/阻力/位于/进一步/阻力/位于/更/关键/阻力/位于/汇价/下行/初步/支撑/位于/进一步/支撑/位于/更/关键/支撑

## 动态主题模型
- gensim：一个python的专门做主题模型的库，它的官方介绍是：Topic modelling for humans

### 利用分好词的文本建立gensim格式的字典
- 字典格式：**{词：索引号}** 即一个词有对应一个索引
- 可以看到字典中一共有14766 个词

In [28]:
from gensim import corpora
from gensim import corpora, models
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
dictionary = corpora.Dictionary(clean_text2)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)
print(dictionary.token2id)

2018-05-20 21:30:30,274 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


Folder "C:\Users\BOKKIN~1\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


2018-05-20 21:30:30,562 : INFO : built Dictionary(14766 unique tokens: [',', ':', '一体化', '一倍', '上']...) from 610 documents (total 345053 corpus positions)
2018-05-20 21:30:30,563 : INFO : saving Dictionary object under C:\Users\BOKKIN~1\AppData\Local\Temp\deerwester.dict, separately None
2018-05-20 21:30:30,573 : INFO : saved C:\Users\BOKKIN~1\AppData\Local\Temp\deerwester.dict


Dictionary(14766 unique tokens: [',', ':', '一体化', '一倍', '上']...)
{',': 0, ':': 1, '一体化': 2, '一倍': 3, '上': 4, '上半年': 5, '下属': 6, '不': 7, '不会': 8, '不到': 9, '业务': 10, '业绩': 11, '中': 12, '中枢': 13, '中美': 14, '中美贸易战': 15, '中长期': 16, '为元': 17, '买入': 18, '事件': 19, '产量': 20, '人': 21, '仍': 22, '仍存': 23, '以来': 24, '低于': 25, '低谷': 26, '供应商': 27, '保护': 28, '倍': 29, '元': 30, '全部': 31, '公司': 32, '公司业绩': 33, '公告': 34, '具备': 35, '再次': 36, '减少': 37, '分别': 38, '北美': 39, '原油': 40, '反弹': 41, '发展': 42, '发布': 43, '受': 44, '受益': 45, '员工': 46, '国内': 47, '国际': 48, '均': 49, '均价': 50, '基础': 51, '增长': 52, '复苏': 53, '多名': 54, '大增': 55, '大幅': 56, '子公司': 57, '已': 58, '带动': 59, '弹性': 60, '强力': 61, '影响': 62, '快速': 63, '成立': 64, '成长': 65, '扩大': 66, '投资': 67, '持续': 68, '控股': 69, '提示': 70, '收入': 71, '改善': 72, '政策': 73, '新高': 74, '日常': 75, '时': 76, '显著': 77, '景气': 78, '最新': 79, '未来': 80, '本土': 81, '本地': 82, '来看': 83, '来自': 84, '极大': 85, '油价': 86, '油价格': 87, '油服': 88, '法律': 89, '波动': 90, '现': 91, '盈利': 92, '目前': 93, '看好': 9

### 利用分好词的文本建立gensim格式的语料库
- 可以看到一共有610条文本
- 语料库格式：**(词索引，词出现的次数)**

In [11]:
corpus = [dictionary.doc2bow(text) for text in clean_text2]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
len(corpus)

610

In [12]:
corpus[0] #第1条新闻文本

[(0, 26),
 (1, 3),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 3),
 (11, 3),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 3),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 3),
 (30, 2),
 (31, 1),
 (32, 17),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 2),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 3),
 (45, 3),
 (46, 1),
 (47, 1),
 (48, 2),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 5),
 (53, 3),
 (54, 1),
 (55, 2),
 (56, 4),
 (57, 2),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 1),
 (62, 4),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 2),
 (69, 1),
 (70, 1),
 (71, 3),
 (72, 1),
 (73, 3),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 2),
 (78, 1),
 (79, 1),
 (80, 3),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 2),
 (86, 4),
 (87, 2),
 (88, 3),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 2),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 3),
 (100, 1

### 建立动态主题模型
- time_slice是一个列表，指定每个时间段的文本数。如下所示，第一个时间段有32篇新闻，第二个时间段有378篇新闻......第五个时间段有31篇新闻。因此语料库按照时间顺序排好很重要

In [48]:
from gensim.models import ldaseqmodel
from gensim import corpora, models,similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time_slice = list(df["period"].value_counts(sort=False))
time_slice

[32, 378, 135, 34, 31]

- ldaseqmodel.LdaSeqModel，建立动态主题模型，即加入时间因素，可使不同主题随着时间变动
- 因为主题为中美贸易战事件，只有一个，设置num_topics=1
- 考虑到运行时间，设置lda迭代次数为15，EM算法迭代次数为10
- 为使不同时间段的主题差异度大一些，设置chain_variance=50

In [14]:
#运行好慢。。。。。大概10分钟
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, 
                                 time_slice=time_slice, num_topics=1,
                                 lda_inference_max_iter=15,
                                 em_max_iter=10,
                                 chain_variance=50)

2018-05-20 20:23:38,884 : INFO : using symmetric eta at 1.0
2018-05-20 20:23:38,887 : INFO : using serial LDA version on this node
2018-05-20 20:23:38,889 : INFO : running online (multi-pass) LDA training, 1 topics, 10 passes over the supplied corpus of 610 documents, updating model once every 610 documents, evaluating perplexity every 610 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-20 20:23:39,647 : INFO : -9.875 per-word bound, 939.3 perplexity estimate based on a held-out corpus of 610 documents with 345053 words
2018-05-20 20:23:39,648 : INFO : PROGRESS: pass 0, at document #610/610
2018-05-20 20:23:39,755 : INFO : topic #0 (0.010): 0.016*"美国" + 0.015*"中国" + 0.008*"市场" + 0.005*"贸易" + 0.004*"上" + 0.004*"经济" + 0.004*"影响" + 0.004*"特朗普" + 0.004*"可能" + 0.004*"中"
2018-05-20 20:23:39,756 : INFO : topic diff=1.177103, rho=1.000000
2018-05-20 20:23:40,549 : INFO : -8.119 per-word bound, 278.0 perplexity estimate based on a held-out corpus of 610 documents with 

2018-05-20 20:28:17,255 : INFO : Computing bound, all times
2018-05-20 20:28:17,731 : INFO : initial sslm bound is 15896011.559176
2018-05-20 20:28:51,198 : INFO : Computing bound, all times
2018-05-20 20:28:51,675 : INFO : iteration 1 iteration lda seq bound is 15896430.335877 convergence is 0.000026
2018-05-20 20:29:23,125 : INFO : Computing bound, all times
2018-05-20 20:29:23,597 : INFO : iteration 2 iteration lda seq bound is 15896814.191755 convergence is 0.000024
2018-05-20 20:29:23,758 : INFO : iteration 3 iteration lda seq bound is 13092816.172339 convergence is 0.000135
2018-05-20 20:29:23,759 : INFO :  EM iter 4
2018-05-20 20:29:23,760 : INFO : E Step
2018-05-20 20:29:23,762 : INFO : using symmetric eta at 1.0
2018-05-20 20:29:23,765 : INFO : using serial LDA version on this node
2018-05-20 20:29:25,754 : INFO : M Step
2018-05-20 20:29:25,755 : INFO : Fitting topic number 0
2018-05-20 20:29:26,418 : INFO : Computing bound, all times
2018-05-20 20:29:26,886 : INFO : initial s

In [42]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

2018-05-20 21:45:10,840 : INFO : collecting document frequencies
2018-05-20 21:45:10,842 : INFO : PROGRESS: processing document #0
2018-05-20 21:45:10,885 : INFO : calculating IDF weights for 610 documents and 14765 features (186414 matrix non-zeros)


In [43]:
lsi_model = models.LsiModel(corpus=corpus_tfidf,id2word=dictionary,
            num_topics=5)

2018-05-20 21:48:14,388 : INFO : using serial LSI version on this node
2018-05-20 21:48:14,389 : INFO : updating model with new documents
2018-05-20 21:48:14,958 : INFO : preparing a new chunk of documents
2018-05-20 21:48:15,011 : INFO : using 100 extra samples and 2 power iterations
2018-05-20 21:48:15,012 : INFO : 1st phase: constructing (14766, 105) action matrix
2018-05-20 21:48:15,042 : INFO : orthonormalizing (14766, 105) action matrix
2018-05-20 21:48:15,260 : INFO : 2nd phase: running dense svd on (105, 610) matrix
2018-05-20 21:48:15,278 : INFO : computing the final decomposition
2018-05-20 21:48:15,280 : INFO : keeping 5 factors (discarding 76.119% of energy spectrum)
2018-05-20 21:48:15,285 : INFO : processed documents up to #610
2018-05-20 21:48:15,286 : INFO : topic #0(5.194): -0.199*"黄金" + -0.164*"大豆" + -0.146*"美元" + -0.099*"美国" + -0.092*"特朗普" + -0.091*"中国" + -0.091*"非农" + -0.089*"金价" + -0.085*"关税" + -0.078*"万吨"
2018-05-20 21:48:15,289 : INFO : topic #1(3.587): 0.362*"黄金

In [44]:
#这就是基于SVD建立的两个主题模型内容
lsiout=lsi_model.print_topics(5)
print (lsiout[0])
print (lsiout[1])
print (lsiout[2])
print (lsiout[3])
print (lsiout[4])

2018-05-20 21:48:19,254 : INFO : topic #0(5.194): -0.199*"黄金" + -0.164*"大豆" + -0.146*"美元" + -0.099*"美国" + -0.092*"特朗普" + -0.091*"中国" + -0.091*"非农" + -0.089*"金价" + -0.085*"关税" + -0.078*"万吨"
2018-05-20 21:48:19,255 : INFO : topic #1(3.587): 0.362*"黄金" + -0.321*"大豆" + 0.188*"美元" + 0.165*"金价" + 0.151*"非农" + -0.129*"万吨" + 0.114*"一线" + -0.106*"豆粕" + 0.105*"均线" + -0.095*"出口"
2018-05-20 21:48:19,257 : INFO : topic #2(3.171): 0.418*"大豆" + 0.204*"万吨" + 0.189*"豆粕" + 0.146*"吨" + -0.134*"中兴" + 0.133*"美豆" + 0.112*"豆油" + -0.108*"特朗普" + 0.102*"巴西" + 0.099*"价格"
2018-05-20 21:48:19,258 : INFO : topic #3(2.772): 0.232*"港股" + 0.210*"板块" + 0.206*"股" + -0.148*"黄金" + 0.131*"创业板" + -0.130*"特朗普" + 0.119*"逾" + 0.119*"报点" + -0.106*"台湾" + 0.105*"个股"
2018-05-20 21:48:19,260 : INFO : topic #4(2.473): -0.293*"盘面" + -0.212*"空单" + -0.194*"呈" + -0.178*"吨" + -0.176*"反弹" + 0.162*"大豆" + -0.157*"伦铜" + -0.156*"抢反弹" + -0.156*"尚存" + -0.152*"位置"


(0, '-0.199*"黄金" + -0.164*"大豆" + -0.146*"美元" + -0.099*"美国" + -0.092*"特朗普" + -0.091*"中国" + -0.091*"非农" + -0.089*"金价" + -0.085*"关税" + -0.078*"万吨"')
(1, '0.362*"黄金" + -0.321*"大豆" + 0.188*"美元" + 0.165*"金价" + 0.151*"非农" + -0.129*"万吨" + 0.114*"一线" + -0.106*"豆粕" + 0.105*"均线" + -0.095*"出口"')
(2, '0.418*"大豆" + 0.204*"万吨" + 0.189*"豆粕" + 0.146*"吨" + -0.134*"中兴" + 0.133*"美豆" + 0.112*"豆油" + -0.108*"特朗普" + 0.102*"巴西" + 0.099*"价格"')
(3, '0.232*"港股" + 0.210*"板块" + 0.206*"股" + -0.148*"黄金" + 0.131*"创业板" + -0.130*"特朗普" + 0.119*"逾" + 0.119*"报点" + -0.106*"台湾" + 0.105*"个股"')
(4, '-0.293*"盘面" + -0.212*"空单" + -0.194*"呈" + -0.178*"吨" + -0.176*"反弹" + 0.162*"大豆" + -0.157*"伦铜" + -0.156*"抢反弹" + -0.156*"尚存" + -0.152*"位置"')


将文章投影到主题空间中

In [45]:
corpus_lsi = lsi_model[corpus_tfidf]
for doc in corpus_lsi:
    print (doc)

[(0, -0.1198732134620885), (1, 0.0058528665893821359), (2, 0.0049736021818143211), (3, 0.063446159105665881), (4, -0.10965775469857456)]
[(0, -0.23775729530173859), (1, -0.14277872425283597), (2, -0.18780911859232227), (3, -0.1472910675829438), (4, -0.033098804249982107)]
[(0, -0.32784195779444003), (1, -0.27386687908150276), (2, 0.42919653108315931), (3, -0.0058041065126932393), (4, 0.05832213140423425)]
[(0, -0.10225983936587628), (1, -0.038902736257471808), (2, -0.042077012970906935), (3, 0.01480135945955556), (4, 0.0075111689014000739)]
[(0, -0.047184051332818056), (1, -0.030335123188605158), (2, -0.078622765437496092), (3, -0.088366802970760217), (4, -0.021432651356467169)]
[(0, -0.14767609198505752), (1, -0.10613740038833522), (2, -0.0091674758183517818), (3, -0.011582338039858234), (4, -0.0017902527175773454)]
[(0, -0.07911274078740059), (1, -0.049027941286648925), (2, -0.10225694625711999), (3, -0.097155480557323107), (4, -0.022245703852731953)]
[(0, -0.1650419280797065), (1, -

In [53]:
#输入一句话，查询属于LSI得到的哪个主题类型，先建立索引：

index = similarities.MatrixSimilarity(lsi_model[corpus])
query = "多家企业协会致信劝阻在特朗普签署总统备忘录的前一周"
query_bow = dictionary.doc2bow(list(jieba.cut(query)))
print (query_bow)
query_lsi = lsi_model[query_bow]
print (query_lsi)

2018-05-20 21:56:02,930 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-05-20 21:56:02,991 : INFO : creating matrix with 610 documents and 5 features


[(228, 1), (272, 1), (332, 1), (425, 1), (448, 1), (539, 1), (623, 1), (1433, 1), (1623, 1), (2286, 1), (2524, 1)]
[(0, -0.27473124623439771), (1, -0.19759643795390436), (2, -0.20796984767965659), (3, -0.15752033191580914), (4, -0.0058578623404194046)]


In [55]:
#比较和第几句话相似，用LSI得到的索引接着做，并排序输出
sims = index[query_lsi]
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])
print (sort_sims)

[(235, 0.99866223), (256, 0.99850404), (123, 0.99698502), (151, 0.99694455), (607, 0.99655604), (606, 0.99653023), (608, 0.99649745), (54, 0.99628878), (335, 0.99585223), (483, 0.99564457), (480, 0.99528724), (517, 0.99514502), (437, 0.99486518), (416, 0.9948284), (548, 0.99470788), (321, 0.99454975), (455, 0.99449629), (540, 0.9943223), (56, 0.9940083), (178, 0.99365044), (356, 0.99349844), (210, 0.99343419), (579, 0.99339914), (292, 0.99321085), (73, 0.99313819), (229, 0.99304336), (345, 0.9927448), (200, 0.99265033), (1, 0.99259722), (31, 0.99214184), (78, 0.99210191), (27, 0.99201238), (593, 0.99198508), (549, 0.99194413), (510, 0.9919048), (460, 0.99172556), (529, 0.99172509), (350, 0.99158633), (103, 0.99158365), (494, 0.99139231), (427, 0.99131954), (127, 0.99112284), (168, 0.99069381), (255, 0.99048603), (351, 0.99045998), (181, 0.9903748), (526, 0.99027216), (129, 0.98954403), (348, 0.98940909), (310, 0.98933899), (317, 0.98918128), (115, 0.98917389), (243, 0.98908031), (269, 

In [17]:
ldaseq.print_topics(time=4)#最近一段时间的主题词

[[('美国', 0.015757361049780514),
  ('中国', 0.015538518953444954),
  ('美元', 0.0072920358929809744),
  ('中美', 0.0063468132648950484),
  ('市场', 0.0061285953739308755),
  ('经济', 0.005397729759952433),
  ('贸易', 0.0049612571433317754),
  ('不', 0.0043043434118460967),
  ('一个', 0.0041576622899701099),
  ('大豆', 0.0037216060797195705),
  ('指数', 0.0037178810962686815),
  ('中', 0.0036473362638356132),
  ('支撑', 0.003646128831097955),
  ('可能', 0.0035021321911048197),
  ('上', 0.0035020812554577918),
  ('特朗普', 0.0034286878930845396),
  ('还', 0.0034270617711108639),
  ('双方', 0.0033547225056038792),
  ('继续', 0.0032092167147697291),
  ('发展', 0.0032088691226110481)]]

- 输出同一个主题在所有时间段的主题词

In [18]:
ldaseq.print_topic_times(topic=0, top_terms=15)

[[('美国', 0.016870796394135416),
  ('中国', 0.014700239508391864),
  ('市场', 0.0065123649635400902),
  ('关税', 0.0055980998371738359),
  ('特朗普', 0.0050270279707832375),
  ('中', 0.0046840806101060214),
  ('上', 0.0043415215854782684),
  ('影响', 0.0043033902121516771),
  ('贸易', 0.0042656000296625749),
  ('企业', 0.0039602240651817489),
  ('大', 0.0037700184008655911),
  ('公司', 0.00369371505877991),
  ('经济', 0.003618105227966701),
  ('产品', 0.0035794931081734758),
  ('不', 0.0035417668548707454)],
 [('中国', 0.01795567379235076),
  ('美国', 0.016781737692609541),
  ('市场', 0.0079110168589284612),
  ('贸易', 0.0062059850711466864),
  ('特朗普', 0.0054021792557372682),
  ('贸易战', 0.0048419908470340991),
  ('上', 0.0047251757278120657),
  ('经济', 0.0046520671672431492),
  ('关税', 0.0046520557215123997),
  ('影响', 0.0042672924214122989),
  ('可能', 0.0041259832446484106),
  ('美元', 0.0040869242990826558),
  ('中', 0.003833763539701053),
  ('不', 0.003707076367611833),
  ('中美', 0.003502404308804127)],
 [('美国', 0.014586840901

In [ ]:
from gensim.models.wrappers.dtmmodel import DtmModel
from gensim.corpora import Dictionary, bleicorpus
import pyLDAvis

# dtm_path = "/Users/bhargavvader/Downloads/dtm_release/dtm/main"
# dtm_model = DtmModel(dtm_path, corpus, time_slice, num_topics=5, id2word=dictionary, initialize_lda=True)
# dtm_model.save('dtm_news')

# if we've saved before simply load the model
dtm_model = DtmModel.load('dtm_news')

### 找出在不同时间段波动较大的词
- 这些词最能体现不同时间段的特点

In [21]:
#保存所有时间段的前2000个主题词
topwords = ldaseq.print_topic_times(topic=0, top_terms=2000) 

#找5个时期，概率波动最大的词
word_pro = {}#保存词各个时间段的概率的字典
for tw in topwords:
    for w,p in tw:
#        print(w,p)
        if w not in word_pro:
            word_pro[w] = [p]
        else:
            word_pro[w].append(p)


word_var = {}#计算词的概率的标准差的字典
for w,p_list in word_pro.items():
    if len(p_list)==5:
        word_var[w]=np.std(p_list)
        
sorted_word_var = sorted(word_var.items(), key = operator.itemgetter(1), reverse=True) #=True 降序排列
sorted_word_var[:50]

[('中国', 0.0024280877417560145),
 ('美元', 0.0022340264199100447),
 ('中美', 0.0018073550769372702),
 ('万吨', 0.0017705105599599847),
 ('大豆', 0.0016580269480580411),
 ('关税', 0.0015634522357383389),
 ('美国', 0.0014653477399400403),
 ('特朗普', 0.0013785773481711089),
 ('价格', 0.0013169926801330299),
 ('贸易战', 0.0013007050460909998),
 ('贸易', 0.0012403281750907043),
 ('芯片', 0.0012004291770208837),
 ('指数', 0.0010930065885543223),
 ('市场', 0.0010896759928995473),
 ('库存', 0.0010568646080702914),
 ('影响', 0.0010071163164050933),
 ('双方', 0.00099031020330752825),
 ('黄金', 0.00098302402331542889),
 ('月份', 0.00097971253230122565),
 ('支撑', 0.00095401730338000573),
 ('产品', 0.0009256826355501064),
 ('吨', 0.00090935358107700271),
 ('公司', 0.00089610976153970279),
 ('国内', 0.00086766631778395845),
 ('问题', 0.0008552550596549719),
 ('基金', 0.00083626365128849803),
 ('一个', 0.00083609559554060926),
 ('还', 0.00082999190352402413),
 ('经济', 0.00081860051359189813),
 ('亿美元', 0.00080658488750959257),
 ('…', 0.000793733859195064

In [22]:
#把波动比较大的前100个词在5个时间段的概率拎出来

var_word = [item[0] for item in sorted_word_var[:100] if len(item[0])>1]

word_pro_df = pd.DataFrame()
for w in var_word:
    onedf = pd.DataFrame({"word":[w],"p":[" ".join([str(p) for p in word_pro[w]])]})
    word_pro_df = word_pro_df.append(onedf)
word_pro_df
word_pro_df.to_csv("word_pro.csv",index=False)

### 以下是我总结的每个时间段重要的主题词
- 可以画出它们的概率随时间变化的折线图，每个时期选几个主题词画一张，反映事态发展
- 词和每个时间段的概率在[word_pro.csv](word_pro.csv)

In [25]:
#第一时期
word_pro["美国商务部"]
word_pro["制裁"]
word_pro["禁令"]
word_pro["限制"]
word_pro["禁止"]
word_pro["商务部"]
word_pro["出口"]
word_pro["技术"]
word_pro["设备"]
word_pro['产品']
word_pro['影响']  
word_pro["协议"]
word_pro['基站']  
word_pro["高通"]




#第二时期
word_pro['基金']
word_pro['影响']   
word_pro['芯片']  
word_pro['估值']  
word_pro["制裁"]
word_pro["行业"]

#第三时期
word_pro['基金']
word_pro['芯片']  
word_pro['技术']  
word_pro['产品']  
word_pro['产业']  
word_pro['华为']  
word_pro['厂商']  
word_pro['国产']
word_pro['核心技术'] 
word_pro["行业"]

#第四     
    
word_pro['经济']
word_pro['发展'] 
word_pro['产业']
word_pro['制造']
word_pro['全球']  
word_pro['出口']
word_pro['投资']

        
#第五时期    
word_pro["特朗普"]
word_pro['发展']
word_pro['经济']
word_pro['中美']
word_pro['问题']
word_pro['技术']
word_pro["恢复"]



[0.00015238312671433206,
 0.00016078697600430139,
 0.00026237285547312005,
 0.00052224307819096318,
 0.00043801273257871725]